In [1]:
from selenium.webdriver import Chrome
import requests
import lxml.html
from selenium.webdriver import ActionChains
import time
from tqdm import tqdm

In [2]:
browser = Chrome()

# 강남구 음식점 url 수집

In [3]:
df = []
rest_url_list = []
for i in range(1,11) :
    url = f'https://www.mangoplate.com/search/강남?keyword=강남&page={i}' # 페이지별로 url을 변경해가면서 수집
    browser.get(url)
    blocks = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div > div > section > div.search-list-restaurants-inner-wrap > ul > li > div > figure > a')
    for block in blocks:
        rest_url_list.append(block.get_attribute('href'))

    rest_url_list = list(set(rest_url_list)) # url 수집 결과 음식점이 중복되게 수집되므로 집합연산으로 중복제거


store_nm = browser.find_element_by_css_selector('div.restaurant_title_wrap > span > h1').text
browser.implicitly_wait(10)
before_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')

more = browser.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]")
action = ActionChains(browser)
action.move_to_element(more).perform()
browser.implicitly_wait(10)

more.click()
time.sleep(1)

after_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')

while 1:
    if len(before_reviews) != len(after_reviews):
        before_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')
        more = browser.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]")
        action = ActionChains(browser)
        action.move_to_element(more).perform()
        browser.implicitly_wait(10)
        more.click()
        time.sleep(1)
        after_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')
    else:
        break
reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')

In [4]:
len(rest_url_list)

200

In [17]:
browser.get(rest_url_list[0])

In [5]:
total =[]

for i in tqdm(rest_url_list):
    browser.get(i)
    try:
        browser.find_element_by_xpath("/html/body/main/div[2]/div/div/div[3]").click()  # 처음 url접근당시 하단의 팝업창 제거
    except:
        print("")
    try:
        browser.find_element_by_xpath("/html/body/main/div/div/div[1]/img").click() # 처음 url접근당시 하단의 팝업창 제거
    except:
        print("")
    try:    
        store_nm = browser.find_element_by_css_selector('div.restaurant_title_wrap > span > h1').text # 음식점명 크롤링
        browser.implicitly_wait(10)
        
        # 더보기 클릭 전 리뷰개수
        before_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')
        
        # 더보기 누르면서 리뷰들 다 보이게 하기
        more = browser.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]") 
        action = ActionChains(browser)
        action.move_to_element(more).perform() # --> 더보기 위치까지 스크롤 내리기
        browser.implicitly_wait(10)

        more.click() # --> 더보기 클릭
        time.sleep(1)
        
        # 더보기 클릭 후 리뷰개수
        after_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a') 


        while 1:
            if len(before_reviews) != len(after_reviews): # 더보기 클릭 전 리뷰수와 더보기 클릭 후 리뷰수가 같으면 반복 중단
                before_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')
                more = browser.find_element_by_xpath("/html/body/main/article/div[1]/div[1]/div/section[3]/div[2]")
                action = ActionChains(browser)
                action.move_to_element(more).perform()
                browser.implicitly_wait(10)
                more.click()
                time.sleep(1)
                after_reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')
            else:
                break
    except:
        print("")
    
    reviews = browser.find_elements_by_css_selector('body > main > article > div.column-wrapper > div.column-contents > div > section.RestaurantReviewList > ul > li > a')

    for review in reviews:
        try:
            dictionary = {}
            text = review.find_element_by_css_selector(".RestaurantReviewItem__ReviewText").text # 리뷰 text 수집
            emotion = review.find_element_by_css_selector(".RestaurantReviewItem__RatingText").text # 리뷰 감정(?) 수집
            dictionary['text'] = text
            dictionary['emotion'] = emotion
            total.append(dictionary)
        except:
            continue
    print(i, len(total))


  0%|                                                                                          | 0/200 [00:00<?, ?it/s]


  0%|▍                                                                               | 1/200 [00:48<2:42:12, 48.91s/it]

https://www.mangoplate.com/restaurants/f-0D3BHzwln6 90





  1%|▊                                                                               | 2/200 [01:57<3:00:29, 54.69s/it]

https://www.mangoplate.com/restaurants/JZM6HCHhwWoN 241




...

https://www.mangoplate.com/restaurants/wvZeJsW78zwk 15392






100%|█████████████████████████████████████████████████████████████████████████████▌| 199/200 [2:31:44<00:47, 47.33s/it]

https://www.mangoplate.com/restaurants/1v6eP2VN2mVS 15397





100%|██████████████████████████████████████████████████████████████████████████████| 200/200 [2:32:18<00:00, 45.69s/it]

https://www.mangoplate.com/restaurants/gxUqUtrD-IAa 15433


In [6]:
import pandas as pd

In [7]:
review = pd.DataFrame(total)

In [9]:
review.to_csv("review.csv", index = False, encoding = 'utf-8')

In [13]:
review['emotion'].value_counts()

맛있다    11972
괜찮다     2553
별로       908
Name: emotion, dtype: int64